<a href="https://colab.research.google.com/github/bgonzalezl/tools-and-previous-code/blob/main/Small_Tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# División de video por fotogramas

In [ ]:
import cv2
import os
import zipfile
from google.colab import files

# Cargar el archivo de video
uploaded = files.upload()  # Sube tu archivo .mp4

# Asegúrate de que el archivo .mp4 está en la carpeta de trabajo actual
video_path = next(iter(uploaded))  # Nombre del archivo cargado

# Crear una carpeta para guardar los frames
output_folder = 'frames'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Abrir el video usando OpenCV
video = cv2.VideoCapture(video_path)

# Verificar si el video fue abierto correctamente
if not video.isOpened():
    print("Error al abrir el video.")
else:
    frame_count = 0
    success, frame = video.read()

    # Extraer frames hasta que no haya más
    while success:
        frame_filename = os.path.join(output_folder, f"frame_{frame_count:04d}.jpg")
        cv2.imwrite(frame_filename, frame)  # Guardar cada frame como un archivo .jpg
        frame_count += 1
        success, frame = video.read()

    video.release()
    print(f"Extracción completada: {frame_count} frames guardados en '{output_folder}'.")

# Crear un archivo ZIP de la carpeta de frames
zip_filename = 'frames.zip'
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    for root, dirs, files in os.walk(output_folder):
        for file in files:
            zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), output_folder))

# Descargar el archivo ZIP (No funciona XD)
#files.download(zip_filename)


# Limpieza de CSV (Solo números)

In [ ]:
import pandas as pd
import re

# Cargar el archivo CSV
archivo_csv = "amazon.csv"  # Reemplázalo con el nombre correcto
df = pd.read_csv(archivo_csv)

# Función para limpiar los valores eliminando símbolos y dejando solo números
def limpiar_valor(valor):
    if isinstance(valor, str):
        # Usar regex para eliminar todo lo que no sea número o punto decimal
        valor_limpio = re.sub(r"[^\d.]", "", valor)
        return valor_limpio
    return valor  # Si no es string, se deja igual

# Aplicar la limpieza a todas las columnas afectadas
columnas_afectadas = ["discounted_price", "actual_price"]  # Ajustar nombres según el CSV
for columna in columnas_afectadas:
    if columna in df.columns:
        df[columna] = df[columna].apply(limpiar_valor)

# Guardar el nuevo archivo limpio
archivo_salida = "ventas_amazon_limpio.csv"
df.to_csv(archivo_salida, index=False)

print(f"Archivo limpio guardado como {archivo_salida}")

# Conversión de archivos de traducción .ts a archivos .csv

In [ ]:
import xml.etree.ElementTree as ET
import csv

def procesar_archivo_ts(archivo_ts, archivo_csv):
    # Crear un objeto ElementTree y analizar el archivo XML
    tree = ET.parse(archivo_ts)
    root = tree.getroot()

    # Crear un conjunto para almacenar traducciones únicas
    traducciones_unicas = set()

    # Crear un archivo CSV para escribir los resultados
    with open(archivo_csv, 'w', newline='', encoding='utf-8') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(['source', 'translation'])  # Encabezados del CSV

        # Iterar sobre las etiquetas 'message' dentro de la estructura XML
        for message in root.iter('message'):
            # Encontrar las etiquetas 'source' y 'translation' dentro de cada 'message'
            source_text = message.find('source').text
            translation_text = message.find('translation').text

            # Verificar si la traducción ya está en el conjunto
            if translation_text not in traducciones_unicas:
                # Agregar la traducción al conjunto
                traducciones_unicas.add(translation_text)

                # Escribir los textos en el archivo CSV
                csv_writer.writerow([source_text, translation_text])

if __name__ == "__main__":
    archivo_ts = "/content/client_es.ts"  # Reemplaza con la ruta de tu archivo .ts
    archivo_csv = "/content/client_es_SinDuplicados.csv"  # Ruta donde se guardará el archivo CSV

    procesar_archivo_ts(archivo_ts, archivo_csv)

# Imagen a ASCII

In [10]:
import numpy as np
import pandas as pd
import math
import PIL.Image

img = PIL.Image.open("/content/images.jpeg")
width, height = img.size
aspect_ratio = height/width
new_width = 500
new_height = aspect_ratio * new_width * 0.55
img = img.resize((new_width, int(new_height)))
img = img.convert('L')
chars = ["@", "-", "#", "%", "*", "!", "+", "_", "$", ",", "."]
pixels = img.getdata()
new_pixels = [chars[pixel//25] for pixel in pixels]
new_pixels = ''.join(new_pixels)

# split string of chars into multiple strings of length equal to new width and create a list
new_pixels_count = len(new_pixels)
ascii_image = [new_pixels[index:index + new_width] for index in range(0, new_pixels_count, new_width)]
ascii_image = "\n".join(ascii_image)
print(ascii_image)

# write to a text file.
with open("sample_ascii_image.txt", "w") as f:
 f.write(ascii_image)

______,,,,,,,,,,......,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,$$_+!*%%%%%%%********%%***!!!***!!***!!+_$$,,,,,,,..,,,,,,,,,,,,,,,,,,,,,,,,,$$$$$$$___+++!!!!!****%%%%%#####%%%%%**********!!!!!!!!++++++++++!!!+_,,.,,,,.,,,,,,,,.,,,,,,,,,,,,$_++!*!!!!!!_$$,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,$$_+!*%%%!+_$$$$$$$$$$$$$$$$$$$$$$$$$,,,,,,,,$$$$$__+!!!!!!++++++___$$$,,,,,$$,,,,,,,,$$$$____+____$$,,.,,,$$$$$_$,,........,,.......
______$,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,....,,,,,,,,,,$$$__+!*%#-###%%*********!!**!!***%%%*!!+_$,,,,.,,,...,,,,,,,,,,,,,,,,,,,,,,,,$$$$$____+++!!!!****%%%%%%%####%%%%%******!!!!!!!!!!!++++++!!!+_$,,,,,,.,,,,,,,,.,,,,,,,..,,,,$_+!!!!!!!+_$$,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,.,,,,,,,,,$$__+++___+++_$,,,$$_____++____$$$$$$$$,,,,,,,$$$$$$$$_++++!!+++_____$$$,,,,,,,,,,,,,,,$$$__________$$,,.,,,$$$$__$$,,,..............

# WebScrapping desde Twitter usando Búsquedas


In [ ]:
!pip3 install git+https://github.com/JustAnotherArchivist/snscrape.git

import snscrape.modules.twitter as sntwitter
import pandas as pd

# Creating list to append tweet data to
attributes_container = []

# Using TwitterSearchScraper to scrape data and append tweets to list
for i,tweet in enumerate(sntwitter.TwitterSearchScraper('sismo mexico since:2022-04-04 until:2023-04-04').get_items()):
    if i>150:
        print("Finalizado")
        break
    else:
      attributes_container.append([tweet.user.username, tweet.user.verified , tweet.date, tweet.likeCount, tweet.replyCount, tweet.sourceLabel, tweet.user.location , tweet.content])

tweets_df = pd.DataFrame(attributes_container, columns=["User", "Is it verified" , "Date Created", "Number of Likes", "Number of replies" , "Source of Tweet", "Location" , "Tweet"])
tweets_df.head(10)

tweets_df.to_csv(r'\tweets_ejemplo.csv', index=False)

# Autómata de Pila

In [6]:
#Gonzalez Lucero Bruno 5BV1
import random
import string
import pydot
from graphviz import Digraph

estados = {'q0', 'q1', 'q2'}
input_simbolos = {'0', '1'}
stack_simbolos = {'$', 'Z'}
transiciones = {
            ('q0', '0', '$'): [('q1', '$Z')],
            ('q1', '0', 'Z'): [('q1', 'ZZ')],
            ('q1', '1', 'Z'): [('q2', '')],
            ('q2', '1', 'Z'): [('q2', '')]
              }
estado_inicio = 'q0'
estado_aceptacion = {'q2'}
stack = ['$']

def evaluar(input_string):
  estado_actual = estado_inicio
  pda_stack = list(stack)
  pda_stack_string = ''.join(pda_stack)
  lista_id = [(estado_actual, input_string, pda_stack_string)]
  bandera = 0

  for simbolo in input_string:
    if (estado_actual, simbolo, pda_stack[-1]) in transiciones:
      bandera = bandera + 1
      transition = transiciones[(estado_actual, simbolo, pda_stack[-1])]
      estado_sig, stack_simbolos = random.choice(transition)
      pda_stack.pop()
      pda_stack.extend(list(stack_simbolos))
      pda_stack_string = ''.join(pda_stack)
      estado_actual = estado_sig
      lista_id.append((estado_actual, input_string, pda_stack_string))
    else:
      break

    if estado_actual in estado_aceptacion and len(pda_stack) == 1 and bandera == len(input_string):
      lista_id.append(('Aceptado', '', ''))

  return lista_id

def animar(lista_id):
    graph = Digraph(comment='PDA Animation')
    graph.node('start', 'Start')
    #graph.node('accept', 'Accept', shape='doublecircle')

    for i, (estado, input_string, stack_string) in enumerate(lista_id):
      label = f'{estado}\n{input_string}\n{stack_string}'
      graph.node(f'node{i}', label)

    for i in range(len(lista_id) - 1):
      graph.edge(f'node{i}', f'node{i+1}')

    graph.edge('start', 'node0')

    graph.render('pda_animation.gv', view=True, format='png')

# Función para generar una cadena aleatoria
def gen_strings(length):
    return ''.join(random.choice('01') for _ in range(length))

user_input = input('Ingrese una cadena o presione Enter para generar una cadena aleatoria: ')
if user_input:
  input_string = user_input.strip()
else:
  length = random.randint(1, 100000)
  input_string = gen_strings(length)

if len(input_string) <= 100000:
  lista_id = evaluar(input_string)
  print('Evaluación del autómata de pila:')
  for estado, input_string, stack_string in lista_id:
    print(f'{estado}\t{input_string}\t{stack_string}')

  if len(input_string) <= 10:
    animar(lista_id)
    print('La animación del autómata de pila se ha guardado en el archivo "pda_animation.gv".')
  else:
    print('La cadena es demasiado larga para animar el autómata de pila.')
else:
      print('La cadena es demasiado larga. El límite máximo es de 100,000 caracteres.')

Ingrese una cadena o presione Enter para generar una cadena aleatoria: 00001111
Evaluación del autómata de pila:
q0	00001111	$
q1	00001111	$Z
q1	00001111	$ZZ
q1	00001111	$ZZZ
q1	00001111	$ZZZZ
q2	00001111	$ZZZ
q2	00001111	$ZZ
q2	00001111	$Z
q2	00001111	$
Aceptado		
La animación del autómata de pila se ha guardado en el archivo "pda_animation.gv".


# Generación de Palíndromos de 1 y 0



In [ ]:
import random

#González Lucero Bruno

def generar_palindromo(longitud,id_pali):

    if longitud == 0: #Regla 1
        id_pali.append(('','Regla 1'))
        return ''
    elif longitud == 1:
        reg = random.choice(['0', '1']) #Regla 2 y 3
        if reg == '0':
          id_pali.append(('0','Regla 2'))
        else:
          id_pali.append(('1','Regla 3'))
        return reg
    else:
        rule = random.choice([2, 3])
        if rule == 2: #Regla 4 y 5
            palindromo = '0' + generar_palindromo(longitud - 2,id_pali) + '0'
            id_pali.append((palindromo,'Regla 4'))
            return palindromo
        elif rule == 3:
            palindromo = '1' + generar_palindromo(longitud - 2,id_pali) + '1'
            id_pali.append((palindromo,'Regla 5'))
            return palindromo

def generar_archivo_palindromo(longitud,id_pali):
    if longitud > 100000:
        print('La longitud máxima permitida es de 100,000 caracteres.')
        return

    archivo = open('palindromo.txt', 'w')
    palindromo = generar_palindromo(longitud,id_pali)
    for palindromo, estado in id_pali:
      archivo.write(f'{palindromo}\t{estado}' + '\n')
    archivo.write('Palíndromo generado: ' + palindromo + '\n')

    archivo.close()
    print('Palindromo generado:' + palindromo)
    print('El palíndromo se ha generado y se ha guardado en el archivo "palindromo.txt".')
    return palindromo

print('Programa Generador de Palíndromos')
id_pali = []
opcion = input('¿Desea ingresar la longitud del palíndromo o generarla automáticamente? (Ingrese "1" para ingresar o "2" para generar automáticamente): ')

if opcion == '1':
    longitud = int(input('Ingrese la longitud del palíndromo: '))
    palindromo = generar_archivo_palindromo(longitud,id_pali)
elif opcion == '2':
    longitud = random.randint(1, 100000)
    palindromo = generar_archivo_palindromo(longitud,id_pali)
else:
    print('Opción inválida. Por favor, ingrese "1" o "2".')